In [ ]:
import pandas as pd
import numpy as  np
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer

In [ ]:
df1 = pd.read_excel(open('D:/PhD_research/NOC_deep_learning/ML_training_data/ML_training_data/Secured_routing_data.xlsx', 'rb'), sheet_name = 'mesh_2vc_pkt_cnt', dtype = float)
df1 = pd.DataFrame(df1.transpose())
y = pd.DataFrame(df1.index, columns=['labels'])
X = df1.set_index(pd.Index([0,1,2,3,4,5,6,7,8,9,10,11]))
df1 = pd.concat([X, y], axis = 1)
df1.shape

In [ ]:
df2 = pd.read_excel(open('D:/PhD_research/NOC_deep_learning/ML_training_data/ML_training_data/Mix_2_secured_training_data.xlsx', 'rb'), sheet_name = 'mesh_2vc_pkt_cnt_mix_2_traffic')
df2 = pd.DataFrame(df2.transpose())
df2 = df2.reset_index()
df2.columns = (list(range(0, 81)))
df2.rename(columns={ df2.columns[80]: "labels" }, inplace = True)
y = df2['labels']
X = df2.drop(['labels'], axis = 1)
X = X.astype(float)
df2 = pd.concat([X, y], axis = 1)
df2.shape

In [ ]:
df = df1.append(df2)
df.reset_index(drop=True, inplace=True)
df_main = df

In [ ]:
target = df['labels']
df = df.drop(['labels'], axis = 1)

In [ ]:
df_main.head()

In [ ]:
df.head()

In [ ]:
def randompacketgen(p = 0.99, n=5000, returnvectorsize = 80):
    # p is probability of success (for us it it roughly 1 - injectionload)
    from scipy.stats import nbinom
    return [int(nbinom.rvs(n, p, loc=0, size=1, random_state=None)//8) for i in range(returnvectorsize)]

In [ ]:
def generate_data():
    frame = []
    for i in range(78):
        x=np.array(randompacketgen())
        x_noise=x.T
        frame.append(x_noise)
    return frame 
    #df.drop(columns=['index'])
    #df['labels'].iloc[78]=y[0]

In [ ]:
def append_data(data, target):   
    gen_with_labels = pd.concat([data, target], axis = 1)
    return gen_with_labels

In [ ]:
def add_to_original(gen_data, df):
    df = pd.concat([df, gen_data], axis = 0)
    return df

In [ ]:
def add_noise(data, df):
    gen_data = pd.DataFrame(data)
    noise_data = df.add(gen_data)
    return noise_data

In [ ]:
# data = generate_data()
# noise_data = add_noise(data, df)
# gen_data = append_data(noise_data, target)
# df_final = add_to_original(gen_data, df_main)

In [ ]:
from tqdm.notebook import trange, tqdm
for i in trange(2000, desc='Generating Data: '):
    data = generate_data()
    noise_data = add_noise(data, df)
    gen_data = append_data(noise_data, target)
    df_main = add_to_original(gen_data, df_main)

In [ ]:
df= df_main

In [ ]:
df.to_excel(r'D:/PhD_research/NOC_deep_learning/ML_training_data/ML_training_data/save_df/mesh_2vc_saved.xlsx')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_excel(open('D:/PhD_research/NOC_deep_learning/ML_training_data/ML_training_data/save_df/mesh_2vc_saved.xlsx', 'rb'))

In [ ]:
df.shape

In [ ]:
y = df['labels']
X = df.drop(['labels'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Feature Selection and Dimensionality Reduction 

In [ ]:
# from sklearn.decomposition import PCA 
# pca = PCA(n_components=16) ## 1 / 2 / 4 / 8 / 16 
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

# from sklearn.feature_selection import VarianceThreshold
# selector = VarianceThreshold(threshold=0.08)
# X_train = selector.fit_transform(X_train)
# X_test = selector.transform(X_test)

corr_matrix = X_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9957)]

print(to_drop)
saved_to_drop = to_drop
X_train = X_train.drop(X_train[to_drop], axis=1)
X_test = X_test.drop(X_test[to_drop], axis=1)

In [ ]:
sc = Normalizer()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
from tensorflow import keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn import preprocessing

In [ ]:
y_train=pd.DataFrame(y_train)

In [ ]:
y_train.columns=['values']

In [ ]:
y_train['values'] = y_train['values'].str.replace(' ', '')

In [ ]:
y_train=pd.DataFrame(y_train)

In [ ]:
y_train=y_train['values'].str.split("+", n = 1, expand = True) 

In [ ]:
y_train.columns=['Zero','One']

In [ ]:
y_train.head()

In [ ]:
y1=pd.get_dummies(y_train['Zero'])

In [ ]:
y2=pd.get_dummies(y_train['One'])

In [ ]:
y1.columns

In [ ]:
y2.columns

In [ ]:
y2['Barnes']=0

In [ ]:
y_train=y1+y2

In [ ]:
y_train=y_train.values.tolist()

In [ ]:
y_train = np.asarray(y_train, dtype=np.int8)

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
y_train

In [ ]:
from tensorflow.keras.optimizers import Adadelta, Adam

In [ ]:
X_train[0].shape

In [ ]:
model = Sequential()
input_shape = X_train[0].shape
model.add(Dense(units=800,input_shape=input_shape,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(500, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(12, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
# fit the model
history=model.fit(X_train, y_train, epochs=50, batch_size=64,validation_split=0.30)

In [ ]:
import tensorflow as tf
tf.keras.losses.categorical_crossentropy

In [ ]:
keras.backend.set_epsilon(1)

# One - Hot encoding Y_test

In [ ]:
y_test=pd.DataFrame(y_test)

In [ ]:
y_test.columns=['values']

In [ ]:
import tensorflow as tf
tf.keras.losses.categorical_crossentropy

In [ ]:
keras.backend.set_epsilon(1)

# One - Hot encoding Y_test

In [ ]:
y_test=pd.DataFrame(y_test)

In [ ]:
y_test.columns=['values']

In [ ]:
y_test.head()

In [ ]:
y_test['values'] = y_test['values'].str.replace(' ' ,'')

In [ ]:
y_test.head()

In [ ]:
y_test=y_test['values'].str.split("+", n = 1, expand = True)

In [ ]:
y_test.columns=['Zero','One']

In [ ]:
y_test.head()

In [ ]:
y1=pd.get_dummies(y_test.Zero)

In [ ]:
y2=pd.get_dummies(y_test.One)

In [ ]:
y2['Barnes']=0

In [ ]:
y_test=y1+y2

In [ ]:
y_test=y_test.values.tolist()

In [ ]:
y_test = np.asarray(y_test, dtype=np.int8)

# Model predicting probabilites and classes

In [ ]:
y_pred_proba=model.predict_proba(X_test)#probabilities
y_pred=model.predict(X_test)
y_pred_classes=model.predict(X_test)#class prediction

In [ ]:
model.predict(X_test)

In [ ]:
y_pred_proba.argsort()[0][10:]

In [ ]:
lis=y_pred_proba.argsort()

In [ ]:
y_pred_proba=lis[:,10:]

In [ ]:
# nb_classes=12
# y_pred_classes1 = np.eye(nb_classes)[y_pred_classes]

In [ ]:
# y_pred_classes1[0]

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred_proba1=y_pred_proba[:][:,0]
#[:3,0]

In [ ]:
y_pred_proba2=y_pred_proba[:][:,1]

In [ ]:
nb_classes=12
one_hot_targets = np.eye(nb_classes)[y_pred_proba1]

In [ ]:
nb_classes=12
one_hot_targets2 = np.eye(nb_classes)[y_pred_proba2]

In [ ]:
one_hot_targets3=one_hot_targets+one_hot_targets2

In [ ]:
count = 0
count1 = 0
for i in range(len(one_hot_targets3)):
    if np.array_equal(one_hot_targets3[i],y_test[i])==True:
        count=count+1
    else:
        pass

In [ ]:
one_hot_targets3[0]

In [ ]:
print(count/len(one_hot_targets3)*100)

In [ ]:
data=pd.DataFrame()

In [ ]:
data.fillna(0)
print(data)

In [ ]:
y_test_1 = y_test
print(y_test_1)

In [ ]:
data['count']=0

In [ ]:
count_list = []
for i in range(len(y_test_1)):
    count=0
    temp = 0
    
    for x in y_test[i]:
        #print("class value:", x)
        if x == 1:
            count=count+1
    var = count
    count_list.append(var)
# print(count_list)


In [ ]:
data['count'] = count_list

In [ ]:
data.head()

In [ ]:
type(y_pred_proba)

for i in range(len(y_pred_proba)):
    if data.iloc[i] is 2:
        y_pred_proba_df[i].append(y_pred_proba[i][10:12])
    else:
        y_pred_proba_df.iloc[i].append(y_pred_proba[i][11:12])

In [ ]:
data1=data[data['count']==1]
data2=data[data['count']==2]

In [ ]:
data1.index

In [ ]:
y_pred_proba

In [ ]:
for i in (data1.index):
    y_pred_proba[i]=y_pred_proba[i][1]

In [ ]:
for i in (data2.index):
    y_pred_proba[i]=y_pred_proba[i]

In [ ]:
y_pred_proba3=y_pred_proba[:][:,0]

In [ ]:
y_pred_proba3

In [ ]:
y_pred_proba4=y_pred_proba[:][:,1]

In [ ]:
y_pred_proba4

In [ ]:
nb_classes=12
one_hot_targets = np.eye(nb_classes)[y_pred_proba3]

In [ ]:
nb_classes=12
one_hot_targets2 = np.eye(nb_classes)[y_pred_proba4]

In [ ]:
one_hot_targets3=one_hot_targets+one_hot_targets2

In [ ]:
np.where(one_hot_targets3>1)

In [ ]:
for i in np.where(one_hot_targets3>1):
    print(one_hot_targets3[i])

In [ ]:
one_hot_targets3[one_hot_targets3==2]=1

In [ ]:
one_hot_targets3[0]

In [ ]:
one_hot_targets3[1]

In [ ]:
print(y_test[0])
print(y_test[1])

In [ ]:
count = 0
count1 = 0
for i in range(len(one_hot_targets3)):
    if np.array_equal(one_hot_targets3[i],y_test[i])==True:
        count=count+1
    else:
        pass

## Accuracy

In [ ]:
print(count/len(one_hot_targets3)*100)

In [ ]:
one_hot_targets3[0]

In [ ]:
one_hot_targets3[1]

In [ ]:
def decode(datum):
    return np.where(datum==1)

In [ ]:
lst_decode=[]

In [ ]:
for i in range(len(one_hot_targets3)):
    datum = one_hot_targets3[i]
    print('index: %d' % i)
    print('encoded datum: %s' % datum)
    lst_decode.append(decode(one_hot_targets3[i]))
    print('decoded datum: %s' % decode(datum))
    print()


In [ ]:
lst_decode_y_test=[]

In [ ]:
for i in range(len(y_test)):
    datum = y_test[i]
    print('index: %d' % i)
    print('encoded datum: %s' % datum)
    lst_decode_y_test.append(decode(y_test[i]))
    print('decoded datum: %s' % decode(datum))
    print()


In [ ]:
lst_decode_df=pd.DataFrame(lst_decode, columns = ['label'])

In [ ]:
lst_decode_df_y_test=pd.DataFrame(lst_decode_y_test, columns = ['label'])

In [ ]:
lst_decode_df.head()

In [ ]:
lst_decode_df_y_test.head()

In [ ]:
y_pred = []
for val in lst_decode_df['label']:
    y_pred.append("-".join(str(x) for x in val))

In [ ]:
y_test = []
for val in lst_decode_df_y_test['label']:
    y_test.append("-".join(str(x) for x in val))

In [ ]:
y_pred = pd.DataFrame(y_pred, columns = ['label'])
y_test = pd.DataFrame(y_test, columns = ['label'])

In [ ]:
y_pred['label'].value_counts()

In [ ]:
class_names=y_test['label'].value_counts().index

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, output_dict=True)
result = pd.DataFrame(report).transpose()
print(result)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
mat=confusion_matrix(y_test,y_pred)
fig,ax=plot_confusion_matrix(conf_mat=mat,figsize=(15,15),class_names=class_names,show_normed=False)
plt.tight_layout()
fig.savefig('cm.png')

In [ ]:
import sklearn

In [ ]:
from sklearn.metrics import classification_report
from sklearn.utils.multiclass import type_of_target

In [ ]:
# y_test = type_of_target(lst_decode_df_y_test['label'])
# y_pred = type_of_target(lst_decode_df['label'])

In [ ]:
print(lst_decode_df['label'])

In [ ]:
report = classification_report((np.hstack(lst_decode_df_y_test['label'])), (np.hstack(lst_decode_df['label'])), output_dict=True )
result = pd.DataFrame(report).transpose()
result

In [ ]:
lst_decode_df=pd.DataFrame(lst_decode_df)

In [ ]:
arr1=np.array(lst_decode_df)

In [ ]:
arr2=np.array(lst_decode_df_y_test)

In [ ]:
arr1[100][0]#[0]

# Testing on Different Dataset with the Pre-trained Model 

In [ ]:
# df3 = pd.read_excel(open('D:/PhD_research/NOC_deep_learning/ML_training_data/ML_training_data/testing_data/Mix_2_mesh_random_routing_data_alpha_0p01.xlsx', 'rb'), sheet_name = 'mesh_2vc_mix_2_traffic')
df3 = pd.read_excel(open('D:/PhD_research/NOC_deep_learning/ML_training_data/ML_training_data/testing_data/Mix_2_mesh_random_routing_data_alpha_0p005.xlsx', 'rb'), sheet_name = 'mesh_2vc_mix_2_traffic')
df3 = pd.DataFrame(df3.transpose())
df3 = df3.reset_index()
df3.columns = (list(range(0, 81)))
df3.rename(columns={ df3.columns[0]: "labels" }, inplace = True)
# y = df3['labels']
# X = df3.drop(['labels'], axis = 1)
# df3 = pd.concat([X, y], axis = 1)
y_sp_test = df3['labels']
X_sp_test = df3.drop(['labels'], axis = 1)
print(y_sp_test.shape)
print(X_sp_test.shape)

In [ ]:
print(saved_to_drop)
print("Columns to drop",len(saved_to_drop))
X_sp_test = X_sp_test.drop(X_sp_test[saved_to_drop], axis=1)

In [ ]:
X_sp_test = sc.transform(X_sp_test)
print(X_sp_test.shape)

# One - Hot encoding Y_test

In [ ]:
y_sp_test

In [ ]:
y_sp_test=pd.DataFrame(y_sp_test)
y_sp_test.columns=['values']
y_sp_test.head()
y_sp_test.shape
y_sp_test['values'] = y_sp_test['values'].str.replace(' ' ,'')
y_sp_test.head()
y_sp_test = y_sp_test['values'].str.split("+", n = 1, expand = True)
y_sp_test.columns=['Zero','One']
y1=pd.get_dummies(y_sp_test.Zero)
#print(y1)
y2=pd.get_dummies(y_sp_test.One)
#print(y2)

y2['Barnes']=0
y1['water_nsquared']=0
# y1.columns
# y2.columns
y_sp_test=y1+y2
y_sp_test=y_sp_test.values.tolist()
y_sp_test = np.asarray(y_sp_test, dtype=np.int8)
#print(y_sp_test)

In [ ]:
y_sp_test

# Model predicting probabilites and classes

In [ ]:
y_sp_pred_proba=model.predict_proba(X_sp_test)
lis=y_sp_pred_proba.argsort()
y_sp_pred_proba=lis[:,10:]
print(y_sp_pred_proba)

In [ ]:
data=pd.DataFrame()
data.fillna(0)
data['count']=0

In [ ]:
count_list = [] 
for i in range(len(y_sp_test)):
    count=0
    temp = 0
    for x in y_sp_test[i]:
        if x == 1:
            count=count+1
    var = count
    count_list.append(var)

In [ ]:
data['count'] = count_list
data1=data[data['count']==1]
data2=data[data['count']==2]

In [ ]:
for i in (data1.index):
    y_sp_pred_proba[i]=y_sp_pred_proba[i][1]

for i in (data2.index):
    y_sp_pred_proba[i]=y_sp_pred_proba[i]

In [ ]:
y_pred_proba3=y_sp_pred_proba[:][:,0]
y_pred_proba4=y_sp_pred_proba[:][:,1]

In [ ]:
nb_classes=12
one_hot_targets = np.eye(nb_classes)[y_pred_proba3]

nb_classes=12
one_hot_targets2 = np.eye(nb_classes)[y_pred_proba4]

one_hot_targets=one_hot_targets+one_hot_targets2

In [ ]:
# for i in np.where(one_hot_targets3>1): #IMP
#     print(one_hot_targets3[i])

In [ ]:
one_hot_targets[one_hot_targets==2]=1

In [ ]:
def decode(datum):
    return np.where(datum==1)

In [ ]:
lst_decode=[]
for i in range(len(one_hot_targets)):
    datum = one_hot_targets[i]
    print('index: %d' % i)
    print('encoded datum: %s' % datum)
    lst_decode.append(decode(one_hot_targets[i]))
    print('decoded datum: %s' % decode(datum))
    print()

In [ ]:
lst_decode_y_test=[]
for i in range(len(y_sp_test)):
    datum = y_sp_test[i]
    print('index: %d' % i)
    print('encoded datum: %s' % datum)
    lst_decode_y_test.append(decode(y_sp_test[i]))
    print('decoded datum: %s' % decode(datum))
    print()

In [ ]:
lst_decode_df=pd.DataFrame(lst_decode, columns = ['label'])
lst_decode_df_y_test=pd.DataFrame(lst_decode_y_test, columns = ['label'])

In [ ]:
# print(lst_decode_df.shape)
# print(lst_decode_df_y_test.shape)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report((np.hstack(lst_decode_df_y_test['label'])), (np.hstack(lst_decode_df['label'])), output_dict=True )
result = pd.DataFrame(report).transpose()
result

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
# accuracy: (tp + tn) / (p + n)
# accuracy = accuracy_score(lst_decode_df_y_test['label'],lst_decode_df['label'])
# print('Accuracy: %f' % accuracy)
# # precision tp / (tp + fp)
# precision = precision_score(testy, yhat_classes)
# print('Precision: %f' % precision)
# # recall: tp / (tp + fn)
# recall = recall_score(testy, yhat_classes)
# print('Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(testy, yhat_classes)
# print('F1 score: %f' % f1)

In [ ]:
lst_decode_df_y_test['label']